Step 1: Import the necessary libraries

In [ ]:
##https://docs.telethon.dev/en/stable/basic/installation.html
import sys
import os
SCRIPT_DIR = os.path.dirname(os.path.abspath(__file__))
sys.path.append(os.path.dirname(SCRIPT_DIR))
from telethon import TelegramClient
from dotenv import load_dotenv
import pandas as pd

Step 2: Authenticate and login via telegram

In [ ]:
# help me import the secrets from .env 
# keys are APP_ID and APP_API_HASH

load_dotenv()
api_id = os.getenv('API_ID')
api_hash = os.getenv('API_HASH')
username = os.getenv('USERNAME')
## disconnect any existing sessions
if f"{username}.session" in os.listdir():
    os.remove(f"{username}.session")
client = TelegramClient(username, api_id, api_hash)
await client.start()
client.connect()

Step 3: Functions

In [ ]:

async def getProfile(json_file_path):
    # Getting information about yourself
    me = await client.get_me()
    user_info = {
    'id': me.id,
    'is_self': me.is_self,
    'contact': me.contact,
    'username': me.username,
    'mutual_contact': me.mutual_contact,
    'deleted': me.deleted,
    'bot': me.bot,
    'bot_chat_history': me.bot_chat_history,
    'bot_nochats': me.bot_nochats,
    'verified': me.verified,
    'restricted': me.restricted,
    'min': me.min,
    'bot_inline_geo': me.bot_inline_geo,
    'support': me.support,
    'scam': me.scam,
    'apply_min_photo':me.apply_min_photo,
    'fake': me.fake,
    'bot_attach_menu':me.bot_attach_menu,
    'premium':me.premium,
    'attach_menu_enabled':me.attach_menu_enabled,
    'bot_can_edit':me.bot_can_edit,
    'close_friend':me.close_friend,
    'stories_hidden':me.stories_hidden,
    'stories_unavailable': me.stories_unavailable,
    'access_hash': me.access_hash,
    'first_name': me.first_name,
    'last_name':me.last_name,
    'username': me.username,
    'phone':me.phone,
    # Add other fields you want to save here
}

In [ ]:
async def getMessages(df,me, users: [str], relationship: str):
   
    for user in users:
        async for message in client.iter_messages(user):
            # Append the message to the DataFrame.
            df = df.append({'id': message.id,'user_id':me.id, 'sender_id': message.sender_id, 'receiver_id': message.peer_id.user_id, 'receiver_name': user, 'text': message.text,'relationship':relationship, 'type': type,'date':message.date}, ignore_index=True)

            print(message)


    return df
    

Step 4. Extract data

In [ ]:
arrayOfFriends=[] # array of friends here: eg ['FriendA','FriendB',...], array cannot be empty
#init dataframe
df = pd.DataFrame(columns=['id','user_id', 'sender_id', 'receiver_id', 'receiver_name', 'text','relationship','type','date'])
## extract messages
me = await client.get_me()
df = await getMessages(df,me,arrayOfFriends,'FRIEND')

In [ ]:
df.to_csv("datasets/raw_messages.csv", sep=",")


Step 5. Exploration of individual text messages

# Get the most frequent occuring text by me

In [ ]:
me = await client.get_me()
df = pd.DataFrame(columns=['id','user_id', 'sender_id', 'receiver_id', 'receiver_name', 'text','relationship','type','date'])
df = await getMessages(df,me,[],'FRIEND') ## your friend telehandle here, array cannot be empty

In [ ]:
import matplotlib.pyplot as plt

# Assuming you already have your DataFrame df

me_df = df[(df['sender_id'] == me.id)& (df['text']!='')]
me_text_occurrences = me_df['text'].value_counts()
me_text_occurrences_filtered = me_text_occurrences[me_text_occurrences > 1]

# Plot a bar plot
plt.figure(figsize=(20, 6))
me_text_occurrences_filtered.plot(kind='bar')
plt.title('Text Occurrences Bar Plot (Counts > 1)')
plt.xlabel('Text')
plt.ylabel('Occurrences')
plt.show()
print("top 10 most popular texts:")

# Count the occurrences of each text


# Get the top 10 most frequent texts
top_10_texts = me_text_occurrences.head(10)

print(top_10_texts)

# Get the most frequent occuring text by reciever

In [ ]:

# Assuming you already have your DataFrame df
rec_df = df[(df['sender_id'] != me.id)& (df['text']!='')]
rec_text_occurrences = rec_df['text'].value_counts()
rec_text_occurrences_filtered = rec_text_occurrences[rec_text_occurrences > 1]

# Plot a bar plot
plt.figure(figsize=(20, 6))
rec_text_occurrences_filtered.plot(kind='bar')
plt.title('Text Occurrences Bar Plot (Counts > 1)')
plt.xlabel('Text')
plt.ylabel('Occurrences')
plt.show()
print("top 10 most popular texts:")

# Count the occurrences of each text


# Get the top 10 most frequent texts
top_10_texts = rec_text_occurrences.head(10)

print(top_10_texts)